<a href="https://colab.research.google.com/github/10100111/Diploma/blob/main/Autokeras_regression_v2_DecisionTreeRegressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Описание задачи

Предсказать, на какой станции и через какой промежуток
времени, этот пассажир воспользуется метро повторно

Описание столбцов для train и test:

● id - уникальный идентификатор столбца;

● ticket_id - уникальный идентификатор
билета, считается, что у одного
билета один владелец

● ticket_type_nm - тип билета

● entrance_id - уникальный id входа в станцию

● entrance_nm - название

● station_id - уникальное id станции захода

● station_nm - наименование станции захода

● line_id - уникальный id ветки на, которой находится станция

● line_nm - наименование ветки, на которой находится станция

● pass_dttm - дата валидации

● time_to_under (столбец для предсказания) - сколько времени прошло
между первой и второй валидацией

● label (столбец для предсказания) - id второй станции, на которой
произошла валидация

# Загрузка библиотек

In [ ]:
!pip install autokeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.8/466.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 22.4 MB/s eta 0:00:00


In [ ]:
import autokeras as ak

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

from sklearn import preprocessing # Для вызова метода .normalize()
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor

from tensorflow.keras import utils # Используем для to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Autokeras regression

In [ ]:
x_train_reg = pd.read_csv('/content/drive/MyDrive/Базы/Metro/Datasets/datasets_df_clf/X_train_reg_df.csv')
x_test_reg = pd.read_csv('/content/drive/MyDrive/Базы/Metro/Datasets/datasets_df_clf/X_test_reg_df.csv')
y_train_reg = pd.read_csv('/content/drive/MyDrive/Базы/Metro/Datasets/datasets_df_clf/Y_train_reg_df.csv')
y_test_reg = pd.read_csv('/content/drive/MyDrive/Базы/Metro/Datasets/datasets_df_clf/Y_test_reg_df.csv')

x_train_reg.shape, x_test_reg.shape, y_train_reg.shape, y_test_reg.shape

((872816, 25), (218205, 25), (872816, 1), (218205, 1))

In [ ]:
# Определение EarlyStopping-коллбека - это остановка обучения, когда НС больше не учится
es_callback = EarlyStopping(monitor='val_loss', patience=4) # останавливаем модель , если алгоритм сошелся

In [ ]:
start_time = time.time()
reg = ak.StructuredDataRegressor(overwrite=True, max_trials=25, tuner = 'bayesian', metrics=['mae', 'mse'],
                         project_name="my_project", directory="my_dir",)
# Обучаем 25  моделей
reg.fit(x_train_reg,
        y_train_reg,
        epochs=15,
        batch_size=128,
        validation_split=0.15,
        callbacks=[es_callback],)
end_time = time.time() # засекаем время
# Вывод времени обучения и точности модели
print("Total training time:", end_time - start_time)
print("Training mae:", reg.evaluate(x_train_reg, y_train_reg)[1])
print("Test mae:", reg.evaluate(x_test_reg, y_test_reg)[1])

# Выведем на экрн лучшую модель
model = reg.export_model()
model.summary()

# Сохраним модель
try:
    model.save("/content/drive/MyDrive/Базы/Metro/autokeras_model_reg_3", save_format="keras")
except Exception:
    model.save("/content/drive/MyDrive/Базы/Metro/autokeras_model_reg_3.h5")


Trial 25 Complete [00h 23m 55s]
val_loss: 13728.91796875

Best val_loss So Far: 908.6651000976562
Total elapsed time: 09h 31m 54s
Epoch 1/15
6819/6819 [==============================] - 79s 11ms/step - loss: 12404.9756 - mae: 83.1891 - mse: 12404.9756
Epoch 2/15
6819/6819 [==============================] - 74s 11ms/step - loss: 6853.7617 - mae: 61.9483 - mse: 6853.7617
Epoch 3/15
6819/6819 [==============================] - 76s 11ms/step - loss: 5579.6406 - mae: 55.9730 - mse: 5579.6406
Epoch 4/15
6819/6819 [==============================] - 77s 11ms/step - loss: 4963.1382 - mae: 52.9476 - mse: 4963.1382
Epoch 5/15
6819/6819 [==============================] - 79s 12ms/step - loss: 4553.2549 - mae: 50.8330 - mse: 4553.2549
Epoch 6/15
6819/6819 [==============================] - 79s 12ms/step - loss: 4313.8105 - mae: 49.5095 - mse: 4313.8105
Epoch 7/15
6819/6819 [==============================] - 78s 11ms/step - loss: 4166.1582 - mae: 48.7259 - mse: 4166.1582
Epoch 8/15
6819/6819 [======

In [ ]:
# Загрузка модели
model = load_model('/content/drive/MyDrive/Базы/Metro/Models/autokeras_model_reg_3', custom_objects=ak.CUSTOM_OBJECTS)

In [ ]:
# Predict with the best model.
expected = y_test_reg
predicted = model.predict(x_test_reg)

6819/6819 [==============================] - 36s 5ms/step


In [ ]:
r2_metric = r2_score(expected, predicted)
r2_metric

0.9464592120942144

In [ ]:
# # The model weights (that are considered the best) are loaded into the
# # model.
# model.load_weights(checkpoint_filepath)

In [ ]:
model.save('/content/autokeras_model_reg_3')

In [ ]:
model_reg = load_model('/content/autokeras_model_reg_3')

## Сохранение модели и проверка результата r2 = 0,94

In [ ]:
# Загрузка модели
model = load_model('/content/drive/MyDrive/Базы/Metro/Models/autokeras_model_reg_3', custom_objects=ak.CUSTOM_OBJECTS)

In [ ]:
# Predict with the best model.
y_pred_reg = model_reg.predict(x_test_reg)

6819/6819 [==============================] - 39s 6ms/step


In [ ]:
r2_metric = r2_score(expected, y_pred_reg)
r2_metric

0.9464592120942144

# DecisionTreeRegressor

In [ ]:
# Объявим модель регрессии с помощью алгоритма решающих деревьев
regressor = DecisionTreeRegressor()
# Вызовем процедуру эмпирического оценивания обобщающей способности алгоритма - кросс валидацию
cross_val_score(regressor, x_test_reg, y_test_reg, cv=10)


array([0.98991299, 0.98972632, 0.98974076, 0.98981963, 0.98966783,
       0.9896663 , 0.98981712, 0.98964026, 0.98971712, 0.98977516])

In [ ]:
# Обучим модель на тренировочных данных
regressor.fit(x_train_reg, y_train_reg,)

DecisionTreeRegressor()


In [ ]:
# Предскажем тестовую выборку на лючшей модели алгоритма
expected_dtr = y_test_reg
predicted_dtr = regressor.predict(x_test_reg)

In [ ]:
# Вызовем метрику r2, которая требуется по условию ТЗ
r2_metric_dtr = r2_score(expected_dtr, predicted_dtr)
r2_metric_dtr

0.9902458149657541

## Сохранение модели и проверка результата r2 = 0,99

In [ ]:
from joblib import dump, load

In [ ]:
# Сохраним файл с моделью
dump(regressor, '/content/drive/MyDrive/Базы/Metro/Models/DecisionTreeRegressor.joblib')
reg = load('/content/drive/MyDrive/Базы/Metro/Models/DecisionTreeRegressor.joblib')

In [ ]:
# Predict with the best model.
y_pred_reg = reg.predict(x_test_reg)

In [ ]:
print(r2_score(y_test_reg, y_pred_reg))


0.9902458149657541
